# Class 8 Notebook 3: SparkSQL

Class 8 (7 Dec 2016) of [BS1804-1617 Fundamentals of Database Technologies](https://imperialbusiness.school/category/bs1804-1617/) by [Piotr Migdal](http://p.migdal.pl/)

References:

* [Spark SQL, DataFrames and Datasets Guide - Spark Programming Guide](http://spark.apache.org/docs/latest/sql-programming-guide.html)
* [PySpark - SQL module](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html)

In [1]:
!wget -P /tmp https://s3.amazonaws.com/pmigdal/city.json
!wget -P /tmp https://s3.amazonaws.com/pmigdal/country.json
!wget -P /tmp https://s3.amazonaws.com/pmigdal/countrylanguage.json

--2016-12-11 20:52:39--  https://s3.amazonaws.com/pmigdal/city.json
Resolving s3.amazonaws.com (s3.amazonaws.com)... 54.231.49.51
Connecting to s3.amazonaws.com (s3.amazonaws.com)|54.231.49.51|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 381063 (372K) [application/octet-stream]
Saving to: ‘/tmp/city.json.1’

/tmp/city.json.1    100%[=====================>] 372.13K   337KB/s   in 1.1s   

2016-12-11 20:52:46 (337 KB/s) - ‘/tmp/city.json.1’ saved [381063/381063]

--2016-12-11 20:52:46--  https://s3.amazonaws.com/pmigdal/country.json
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.0.139
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.0.139|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75715 (74K) [application/octet-stream]
Saving to: ‘/tmp/country.json.1’

/tmp/country.json.1 100%[=====================>]  73.94K   201KB/s   in 0.4s   

2016-12-11 20:52:53 (201 KB/s) - ‘/tmp/country.json.1’ saved [75715/75715]

--201

In [9]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext

conf = SparkConf().setAppName("building a warehouse")
sc = SparkContext(conf=conf)
sqlCtx = SQLContext(sc)


# loading data in SparkSQL
city = sqlContext.read.json("file:/tmp/city.json")
# we need to register a table if we want to use global queries
# vide: http://blog.revolutionanalytics.com/2016/05/an-object-has-no-name.html
city.registerTempTable("city")
# schema (thanks to JSON)
city.printSchema()

NameError: name 'sqlContext' is not defined

In [4]:
country = sqlContext.read.json("file:/tmp/country.json")
country.registerTempTable("country")
country.printSchema()

In [5]:
countrylanguage = sqlContext.read.json("file:/tmp/countrylanguage.json")
countrylanguage.registerTempTable("countrylanguage")
countrylanguage.printSchema()

In [6]:
# and we can perform normal queries!
query = """
SELECT
  countrycode,
  COUNT(*) as cities
FROM city
GROUP BY countrycode
ORDER BY COUNT(*) DESC
LIMIT 5
"""
sqlContext.sql(query).toPandas()

In [7]:
# there is also another syntax, more Python-like:
city.select('name').limit(10).show()

In [8]:
# also, we can convert the results to Pandas
# (instead of showing it)
city.select('name').limit(10).toPandas()

## Exercises

* What is the total population per continent (use `country`, not `city`)?
* In which countries in Africa people speak French (>5%)?
* What are the biggest cities (population, globally) in countries where people speak English (>5%)?

In [10]:
query = """
SELECT
  continent,
  SUM(population) as population
FROM
  country
GROUP BY continent
"""
sqlContext.sql(query).show()

In [11]:
query = """
SELECT
  country.name,
  country.population,
  cl.percentage
FROM
  country
INNER JOIN
  countrylanguage AS cl ON cl.countrycode = country.code
WHERE
  cl.language = 'French'
  AND country.continent = 'Africa'
  AND cl.percentage > 0.05
ORDER BY
  cl.percentage DESC
"""
sqlContext.sql(query).show()